<a href="https://colab.research.google.com/github/akadas112/AI_code/blob/main/word_embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
print(tf.__version__)

2.3.0


In [ ]:
from tensorflow.keras import layers

embedding_layer = layers.Embedding(1000, 5)

In [ ]:
result = embedding_layer(tf.constant([1, 2, 3]))
result.numpy()

array([[-0.02137107, -0.04152719, -0.04998957, -0.03166902, -0.02381582],
       [-0.00951926,  0.02617545, -0.03611697, -0.04965253,  0.00415894],
       [ 0.00240667,  0.04115004,  0.03892788, -0.04951764,  0.0241648 ]],
      dtype=float32)

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer()

sentence = ['আমাদের বলতে লজ্জা লাগে', 'আবার আমরা অনেকে এইটাই জানি নাহ যে আমরা কি জানি নাহ!']

tokenizer.fit_on_texts(sentence)
sequences = tokenizer.texts_to_sequences(sentence)

sequences

[[4, 5, 6, 7], [8, 1, 9, 10, 2, 3, 11, 1, 12, 2, 3]]

In [ ]:
result = embedding_layer(tf.constant(sequences[0]))
result.numpy()

array([[ 0.03654111,  0.0071669 ,  0.04394806,  0.00060445,  0.0143384 ],
       [ 0.02666905,  0.02519185, -0.02872103,  0.00645044, -0.021288  ],
       [-0.01814824, -0.00217323, -0.00200601,  0.02258152, -0.04361961],
       [ 0.03025539,  0.01609306, -0.00868966, -0.00676529, -0.00610004]],
      dtype=float32)

In [ ]:
from numpy import array
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import  pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Embedding, Dense, LSTM, Bidirectional

In [ ]:
# Real Life Example of Classification

train_doc = [  'এই হোটেল এর খাবার একদম ফালতু',
               'আমি আর জীবনেও এইখানে খেতে আসব না!',
               'কর্মচারী গুলা একদম বেয়াদব, কথা শুনে নাহ',
               'খাবার গুলা প্রায়ই ঠাণ্ডা থাকে',
               'মাঝে মদ্ধে বাসি খাবারও দেয়',
               'খুবই মজার খাবার!!',
               'আমার ওদের সার্ভিস ভালো লেগেছে',
               'পরিবেশ টি অনেক সুন্দর',
               'তাদের খাবার টাটকা ও সুস্বাদু',
               'তাদের খাবারের দাম অনেক কম']

# Reviews -- negative = 0; positive = 1 (Class/Labels)
train_label = array([0, 0, 0, 0, 0, 1, 1, 1, 1, 1])

In [ ]:
train_doc[4]

'মাঝে মদ্ধে বাসি খাবারও দেয়'

In [ ]:
# tokenization and converting words into sequence
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_doc)
dense_train_doc = tokenizer.texts_to_sequences(train_doc)

dense_train_doc

[[6, 7, 8, 1, 2, 9],
 [10, 11, 12, 13, 14, 15, 16],
 [17, 3, 2, 18, 19, 20, 21],
 [1, 3, 22, 23, 24],
 [25, 26, 27, 28, 29],
 [30, 31, 1],
 [32, 33, 34, 35, 36],
 [37, 38, 4, 39],
 [5, 1, 40, 41, 42],
 [5, 43, 44, 4, 45]]

In [ ]:
# padding the training documents in order to make them equal length.
MAX_LENGTH = 8

padded_trai_doc = pad_sequences(dense_train_doc, maxlen=MAX_LENGTH, padding='post')
padded_trai_doc

array([[ 6,  7,  8,  1,  2,  9,  0,  0],
       [10, 11, 12, 13, 14, 15, 16,  0],
       [17,  3,  2, 18, 19, 20, 21,  0],
       [ 1,  3, 22, 23, 24,  0,  0,  0],
       [25, 26, 27, 28, 29,  0,  0,  0],
       [30, 31,  1,  0,  0,  0,  0,  0],
       [32, 33, 34, 35, 36,  0,  0,  0],
       [37, 38,  4, 39,  0,  0,  0,  0],
       [ 5,  1, 40, 41, 42,  0,  0,  0],
       [ 5, 43, 44,  4, 45,  0,  0,  0]], dtype=int32)

In [ ]:
# Model Declaration
VOCUB_SIZE = 50
model = Sequential()

# embedding layer
embedding_layer = Embedding(VOCUB_SIZE, 7, input_length=MAX_LENGTH)
model.add(embedding_layer)

# Flattening Layer
# model.add(Flatten())

# LSTM Extra - for performance
model.add(LSTM(units=128))

# Bidirectional LSTM Extra
# forward_layer = LSTM(128, return_sequences=False)
# backward_layer = LSTM(128, return_sequences=False, go_backwards=True)
# model.add(Bidirectional(forward_layer, backward_layer=backward_layer))

# output layer
model.add(Dense(units=1, activation='sigmoid'))

model.compile(optimizer='adam', loss='mse', metrics=['acc'])

print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 8, 7)              350       
_________________________________________________________________
lstm (LSTM)                  (None, 128)               69632     
_________________________________________________________________
dense (Dense)                (None, 1)                 129       
Total params: 70,111
Trainable params: 70,111
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
model.fit(padded_trai_doc, train_label, epochs=100, verbose=1)

Epoch 1/100
1/1 [==============================] - 0s 2ms/step - loss: 0.2502 - acc: 0.5000
Epoch 2/100
1/1 [==============================] - 0s 3ms/step - loss: 0.2499 - acc: 0.5000
Epoch 3/100
1/1 [==============================] - 0s 1ms/step - loss: 0.2496 - acc: 0.6000
Epoch 4/100
1/1 [==============================] - 0s 2ms/step - loss: 0.2492 - acc: 0.5000
Epoch 5/100
1/1 [==============================] - 0s 1ms/step - loss: 0.2488 - acc: 0.8000
Epoch 6/100
1/1 [==============================] - 0s 2ms/step - loss: 0.2483 - acc: 1.0000
Epoch 7/100
1/1 [==============================] - 0s 4ms/step - loss: 0.2477 - acc: 1.0000
Epoch 8/100
1/1 [==============================] - 0s 1ms/step - loss: 0.2470 - acc: 1.0000
Epoch 9/100
1/1 [==============================] - 0s 1ms/step - loss: 0.2461 - acc: 1.0000
Epoch 10/100
1/1 [==============================] - 0s 1ms/step - loss: 0.2451 - acc: 1.0000
Epoch 11/100
1/1 [==============================] - 0s 2ms/step - loss: 0.2438 

In [ ]:
test_doc = ['হোটেল টার খাবার ফালতু', 'ভালো লাগার অপর নাম এই রেস্টুরেন্ট', 'বদমাইশ কর্মচারী, পাত্তা দেয় না কাস্টোমারদের']

# tokenization and converting words into sequence
dense_test_doc = tokenizer.texts_to_sequences(test_doc)

# padding the test documents.
padded_test_doc = pad_sequences(dense_test_doc, maxlen=MAX_LENGTH, padding='post')

prediction = model.predict(padded_test_doc)

print("Raw Prediction (Sigmoid Output):\n{}\n".format(prediction))


# making a decision
bin_prediction = []
for [pred] in prediction:
  if pred < 0.5:
    bin_prediction.append(0)   # put 0 if the probability is less than 0.5
  else:
    bin_prediction.append(1)   # 1 if prediction >= 0.5

# Making the Prediction beautiful!! :p

class_type = {0: 'negative', 1: 'positive'}

for indx, doc in enumerate(test_doc):
  print("Test Doc-{}: Prediction = {} ({}) -- {}".format(indx, bin_prediction[indx], class_type[bin_prediction[indx]], doc))

# loss, accuracy = model.evaluate(padded_trai_doc, train_label, )

Raw Prediction (Sigmoid Output):
[[1.9353628e-04]
 [9.9906254e-01]
 [1.2564659e-04]]

Test Doc-0: Prediction = 0 (negative) -- হোটেল টার খাবার ফালতু
Test Doc-1: Prediction = 1 (positive) -- ভালো লাগার অপর নাম এই রেস্টুরেন্ট
Test Doc-2: Prediction = 0 (negative) -- বদমাইশ কর্মচারী, পাত্তা দেয় না কাস্টোমারদের
